### 卷积网络
- 卷积网络
- 互相关运算
- 卷积核/过滤器, 不一定是正方形
- 卷积窗口
- 训练: 迭代卷积核和偏差
- 如何通过数据学习卷积核:
    - 使用gluon自带的Conv2D
        - 参数: 
            - 通道数
            - 核形状
        - 二维卷积层使用4维输入输出，格式为(样本, 通道, 高, 宽): conv2d((samples, channels, height, width))
    - 初始化成数组
    - 计算 损失函数的梯度来更新权重和偏差
- 特征图, feature map: 二维卷积层 的输出 可以看做是输入在空间维度(宽和高)上的某一级的表征
- 感受野, receptive field: 影响 元素x的前向计算的所有可能输入区域(可能会超出输入区域) -> 二维的八个方位
    - 可以通过更深的卷积神经网络使特征图中单个元素的感受野变得更加广阔，从而捕捉输入上更大尺寸的特征
    - 比如 X_3*3 -(K1_2*2)-> Y_2*2 -(K2_2*2)-> Z_1*1,  Z在Y上的感受野是Y的全部4个元素, 而在输入(X)上的感受野是输入的全部9个元素
- 怎么设计卷积核, 令其能够检测出边缘:


In [1]:
from mxnet.gluon import nn
from mxnet import autograd, nd

In [2]:
def corr2d(X, K):
    """
    too slow
    X: input
    K: kernel
    """
    h, w = K.shape
    Y = nd.zeros((X.shape[0] - h + 1, X.shape[1] - w + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i, j] = (X[i: i + h, j: j + w] * K).sum()
    return Y

In [3]:
X = nd.random.uniform(shape=(100, 100))

In [4]:
k = nd.ones(shape=(2, 2)) * 2

In [7]:
X


[[0.5488135  0.5928446  0.71518934 ... 0.45720482 0.55868983 0.43069857]
 [0.14038694 0.9391278  0.19810149 ... 0.12132858 0.5108276  0.5693113 ]
 [0.89294696 0.43706194 0.8962931  ... 0.01902474 0.9489773  0.03978036]
 ...
 [0.9162014  0.44308117 0.59071505 ... 0.5811341  0.05543189 0.99660516]
 [0.49628252 0.6452971  0.80752385 ... 0.9595277  0.6766825  0.50212663]
 [0.6899407  0.47096756 0.13747388 ... 0.6698337  0.7846096  0.26094997]]
<NDArray 100x100 @cpu(0)>

In [8]:
0.5488135 * 2 + 0.5928446 * 2 + 0.14038694 * 2 + 0.9391278 * 2

4.44234568

In [10]:
%time corr2d(X, k)

CPU times: user 2.25 s, sys: 190 ms, total: 2.44 s
Wall time: 2.13 s



[[4.4423456 4.8905263 5.071892  ... 2.5734859 3.2961016 4.139055 ]
 [4.8190475 4.941169  3.7830637 ... 1.7348094 3.2003164 4.137793 ]
 [6.1226377 6.289296  4.4553595 ... 3.1869245 3.990566  3.8759232]
 ...
 [6.3993626 4.4471655 3.894683  ... 3.9462676 4.67764   4.0472145]
 [5.0017242 4.973234  5.442731  ... 5.556615  4.5455523 4.4616923]
 [4.6049757 4.1225247 3.9016984 ... 6.3918395 6.181307  4.448737 ]]
<NDArray 99x99 @cpu(0)>

In [13]:
class Conv2D(nn.Block):
    """
    二维卷积层
    """
    def __init__(self, kernel_shape, **kwargs):
        """
        :param kernel_shape: tuple
        """
        super(Conv2D, self).__init__(**kwargs)
        self.weight = self.params.get("weight", shape=kernel_shape)
        self.bias = self.params.get("bias", shape=(1, ))
    
    def forward(self, X):
        return corr2d(X, self.weight.data()) + self.bias.data()

In [14]:
conv2d = Conv2D((3,3))
conv2d.initialize()
conv2d(X)


[[0.11220663 0.1650652  0.09899607 ... 0.13774395 0.08986556 0.11184095]
 [0.10304052 0.1554206  0.13556352 ... 0.13074315 0.10483424 0.10050067]
 [0.17754516 0.11567509 0.13443696 ... 0.06925815 0.1603241  0.06498009]
 ...
 [0.13761944 0.16539177 0.0734662  ... 0.12492651 0.09096377 0.13385639]
 [0.1245357  0.13291404 0.14092755 ... 0.1288637  0.13798285 0.07410178]
 [0.13239706 0.12881969 0.17085703 ... 0.10291093 0.10774222 0.19160858]]
<NDArray 98x98 @cpu(0)>

In [15]:
conv2d.weight.data()


[[ 0.01637076 -0.01589154  0.06212472]
 [ 0.05636378  0.02545484 -0.007007  ]
 [-0.0196689   0.01582889 -0.00881553]]
<NDArray 3x3 @cpu(0)>

In [16]:
conv2d.bias.data()


[0.0563288]
<NDArray 1 @cpu(0)>

### 检测图片中物体的边缘

In [17]:
images = nd.ones((6, 8))
images[:, 2:6] = 0
images
# 从白到黑, 又从黑到白, 两条分界线


[[1. 1. 0. 0. 0. 0. 1. 1.]
 [1. 1. 0. 0. 0. 0. 1. 1.]
 [1. 1. 0. 0. 0. 0. 1. 1.]
 [1. 1. 0. 0. 0. 0. 1. 1.]
 [1. 1. 0. 0. 0. 0. 1. 1.]
 [1. 1. 0. 0. 0. 0. 1. 1.]]
<NDArray 6x8 @cpu(0)>

In [20]:
K = nd.array([[1, -1]])
K


[[ 1. -1.]]
<NDArray 1x2 @cpu(0)>

In [21]:
images_edge = corr2d(images, K)

In [22]:
images_edge
# 检测出 黑到白的分界线: 1
# 检测出 白到黑的分极限: -1


[[ 0.  1.  0.  0.  0. -1.  0.]
 [ 0.  1.  0.  0.  0. -1.  0.]
 [ 0.  1.  0.  0.  0. -1.  0.]
 [ 0.  1.  0.  0.  0. -1.  0.]
 [ 0.  1.  0.  0.  0. -1.  0.]
 [ 0.  1.  0.  0.  0. -1.  0.]]
<NDArray 6x7 @cpu(0)>

In [23]:
def get_net():
    conv2d = nn.Conv2D(1, kernel_size=(1, 2))
    conv2d.initialize()       
    return conv2d

In [34]:
# 二维卷积层使用4维输入输出，格式为(样本, 通道, 高, 宽)，这里批量大小（批量中的样本数）和通道数均为1
def train_cnn(net, features, labels):
    for i in range(10):
        with autograd.record():
            Y_hat = net(features)
            l = (Y_hat - labels) ** 2
        l.backward()
        # 简单起见，这里忽略了偏差
        # conv2d.weight.data()[:] -= 3e-2 * conv2d.weight.grad()
        net.weight.set_data(net.weight.data() - 3e-2 * net.weight.grad())
        if (i + 1) % 2 == 0:
            print('batch %d, loss %.3f' % (i + 1, l.sum().asscalar()))

In [36]:
net = get_net()
train_cnn(net, features=images.reshape((1, 1, 6, 8)), labels=images_edge.reshape((1, 1, 6, 7)))

batch 2, loss 4.582
batch 4, loss 0.770
batch 6, loss 0.130
batch 8, loss 0.022
batch 10, loss 0.004


In [28]:
images


[[1. 1. 0. 0. 0. 0. 1. 1.]
 [1. 1. 0. 0. 0. 0. 1. 1.]
 [1. 1. 0. 0. 0. 0. 1. 1.]
 [1. 1. 0. 0. 0. 0. 1. 1.]
 [1. 1. 0. 0. 0. 0. 1. 1.]
 [1. 1. 0. 0. 0. 0. 1. 1.]]
<NDArray 6x8 @cpu(0)>

In [29]:
images_edge


[[ 0.  1.  0.  0.  0. -1.  0.]
 [ 0.  1.  0.  0.  0. -1.  0.]
 [ 0.  1.  0.  0.  0. -1.  0.]
 [ 0.  1.  0.  0.  0. -1.  0.]
 [ 0.  1.  0.  0.  0. -1.  0.]
 [ 0.  1.  0.  0.  0. -1.  0.]]
<NDArray 6x7 @cpu(0)>

In [30]:
net.weight.set_data


[[[[ 0.02766836 -0.05610075]]]]
<NDArray 1x1x1x2 @cpu(0)>

In [31]:
net.bias.data()


[0.]
<NDArray 1 @cpu(0)>

### 练习
1. 构造一个输入图像X，令它有水平方向的边缘。如何设计卷积核K来检测图像中水平边缘? 如果是对角方向的边缘呢?
2. 试着对我们自己构造的Conv2D类进行自动求梯度，会有什么样的错误信息？在该类的forward函数里，将corr2d函数替换成nd.Convolution类使得自动求梯度变得可行。
    - 对一个for循环如何求梯度
3. 如何通过变化输入和核数组将互相关运算表示成一个矩阵乘法？
4. 如何构造一个全连接层来进行物体边缘检测？
